# Train and Tune the Models

In [1]:
import os

import optuna

from src.config import Config
from src.data import time_series_split
from src.models.classical import LinearElasticNet
from src.preprocessing import get_preprocessor
from src.train import ModelTrainer
from src.utils import load_csv, set_seed

from pathlib import Path

In [2]:
os.environ["WANDB_MODE"] = "offline"

In [3]:
cfg = Config("../config/config.yaml")
rng = set_seed(cfg.runtime.seed)

In [4]:
df_full = load_csv(str(os.path.join(cfg.data.processed_dir, cfg.data.name_features_full)))
df_ml = df_full.drop(columns=["open", "high", "low", "close", "volume", "adj_close"]).copy()
df_ml

,date,pos,neu,neg,pos_minus_neg,emb_0,emb_1,emb_2,emb_3,emb_4,...,sma_25,ema_25,lag_50,sma_50,ema_50,quarter,dow,q_mean,q_std,q_skew
0,2008-08-08,0.723097,0.080308,0.196594,0.526503,-0.482626,-0.328239,-0.636478,0.627401,0.294770,...,NaN,NaN,NaN,NaN,NaN,3,4,0.000048,0.012294,-0.837789
1,2008-08-11,0.695768,0.168894,0.135338,0.560430,-0.566419,-0.423188,-0.652022,0.812345,0.474358,...,NaN,NaN,NaN,NaN,NaN,3,0,0.000048,0.012294,-0.837789
2,2008-08-12,0.696532,0.040929,0.262539,0.433994,-0.631328,-0.305491,-0.709601,0.736141,0.427515,...,NaN,NaN,NaN,NaN,NaN,3,1,0.000048,0.012294,-0.837789
3,2008-08-13,0.833299,0.003527,0.163175,0.670124,-0.369901,-0.184175,-0.789519,0.645993,0.350836,...,NaN,NaN,NaN,NaN,NaN,3,2,0.000048,0.012294,-0.837789
4,2008-08-14,0.928612,0.022426,0.048962,0.879650,-0.482822,-0.254288,-0.716074,0.775566,0.416154,...,NaN,NaN,NaN,NaN,NaN,3,3,0.000048,0.012294,-0.837789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,2016-06-27,0.684537,0.065429,0.250034,0.434503,-0.126545,-0.069503,-0.505809,0.675737,0.537852,...,-0.000833,-0.002734,-0.001617,-0.000865,-0.001261,2,0,0.000062,0.009649,-0.181613
1985,2016-06-28,0.714076,0.086932,0.198991,0.515085,-0.035628,0.026517,-0.447723,0.623409,0.638349,...,-0.000191,-0.001324,0.005944,-0.000671,-0.000600,2,1,0.000062,0.009649,-0.181613
1986,2016-06-29,0.673349,0.114712,0.211939,0.461410,-0.183834,-0.045571,-0.742004,0.678196,0.518598,...,-0.000026,0.000027,0.002742,-0.000402,0.000060,2,2,0.000062,0.009649,-0.181613
1987,2016-06-30,0.727080,0.040575,0.232345,0.494736,-0.142834,-0.103121,-0.675863,0.680667,0.464056,...,0.000175,0.001041,0.002361,-0.000185,0.000576,2,3,0.000062,0.009649,-0.181613


In [5]:
train, val, test, forecast = time_series_split(df_ml, train_ratio=0.8, val_ratio=0.1, horizon=30)

In [6]:
X_train, y_train = train.drop(columns=["date", "target"]), train["target"]
X_val, y_val = val.drop(columns=["date", "target"]), val["target"]
X_test, y_test = test.drop(columns=["date", "target"]), test["target"]
X_forecast = forecast.drop(columns=["date", "target"])

In [7]:
model = LinearElasticNet(horizon=30)
preprocessor, _ = get_preprocessor(X_train)

config = {
    "model": "linear_elasticnet",
    "optimization_metric": "rmse"
}

trainer = ModelTrainer(
    model=model,
    name="linear_elasticnet",
    config=config,
    preprocessor=preprocessor,
    output_path=cfg.data.models_dir
)

In [9]:
from optuna.integration.wandb import WeightsAndBiasesCallback

opt_metric = config.get("optimization_metric", "rmse")

wandb_callback = WeightsAndBiasesCallback(
    metric_name=opt_metric,
    wandb_kwargs={
        "project": "stock_forecasting",
        "name": "optuna_tuning_run",
        "tags": ["optuna", "linear_elasticnet"],
        "dir": Path("../data/models").resolve()
    }
)

C:\Users\Vito\AppData\Local\Temp\ipykernel_13856\3329073828.py:5: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandb_callback = WeightsAndBiasesCallback(


In [10]:
study = optuna.create_study(direction="minimize")
study.optimize(
    lambda trial: trainer.objective(trial, X_train, y_train, n_splits=5),
    n_trials=30,
    callbacks=[wandb_callback]
)

best_params = study.best_params
print("Best params:", best_params)

[I 2025-08-17 09:45:19,885] A new study created in memory with name: no-name-3627eab0-59c3-4ee0-ae33-08994d7d0662
2025-08-17 09:45:19,886 - INFO - ModelTrainer - Running Optuna trial with params: {'alpha': 1.4661896418400024, 'l1_ratio': 0.9398973231547251}
2025-08-17 09:45:19,913 - INFO - LinearElasticNet - Starting model training...
2025-08-17 09:45:19,923 - INFO - LinearElasticNet - Starting model training...
2025-08-17 09:45:19,932 - INFO - LinearElasticNet - Starting model training...
2025-08-17 09:45:19,941 - INFO - LinearElasticNet - Starting model training...
2025-08-17 09:45:19,954 - INFO - LinearElasticNet - Starting model training...
[I 2025-08-17 09:45:19,956] Trial 0 finished with value: 0.008936553071680106 and parameters: {'alpha': 1.4661896418400024, 'l1_ratio': 0.9398973231547251}. Best is trial 0 with value: 0.008936553071680106.
2025-08-17 09:45:19,957 - INFO - ModelTrainer - Running Optuna trial with params: {'alpha': 0.015166679968592289, 'l1_ratio': 0.234131148949

Best params: {'alpha': 1.4661896418400024, 'l1_ratio': 0.9398973231547251}
Best trial: {'alpha': 1.4661896418400024, 'l1_ratio': 0.9398973231547251}


In [11]:
best_model = LinearElasticNet(horizon=30, **best_params)
trainer = ModelTrainer(best_model, name="lin_reg_best", config=best_params, preprocessor=preprocessor)
trainer.fit(X_train, y_train)

2025-08-17 09:45:38,319 - INFO - ModelTrainer - Starting model training...
2025-08-17 09:45:38,326 - INFO - LinearElasticNet - Starting model training...


In [12]:
eval_metrics = trainer.evaluate(X_test, y_test)
print(eval_metrics)

2025-08-17 09:45:40,049 - INFO - ModelTrainer - Evaluating model...


{'mae': 0.007272825545509099, 'mse': 9.133358088617656e-05, 'rmse': 0.009556860409474262, 'smape': 1.8163904321451736, 'r2': -0.00046888392077071295}


In [16]:
import optuna.visualization as vis

vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_contour(study)
vis.plot_parallel_coordinate(study)

In [17]:
model_path = trainer.save()
trainer.track_wandb(metrics=eval_metrics, model_path=model_path)

mae,▁
mse,▁
r2,▁
rmse,▁
smape,▁
mae,0.00727
mse,9e-05
r2,-0.00047
rmse,0.00956
smape,1.81639


mae,▁
mse,▁
r2,▁
rmse,▁
smape,▁
mae,0.00727
mse,9e-05
r2,-0.00047
rmse,0.00956
smape,1.81639


In [ ]:
import numpy as np


print("Target variance:", np.var(y_test))

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt


def plot_forecast_from_predictions(
        full_df: pd.DataFrame,
        forecast_df: pd.DataFrame,
        predicted_returns: np.ndarray,
        price_col: str = "adj_close",
        forecast_col: str = "Forecasted Price",
        title: str = "Forecasted Prices vs Actual",
        zoom_days: int = 180,
        figsize=(14, 6)
):
    """
    Plot actual vs forecasted prices using predicted log returns from the model.

    Parameters:
    - full_df: The complete original DataFrame with prices.
    - forecast_df: The subset used for forecasting (e.g., last horizon days).
    - predicted_returns: Model's predicted log returns.
    - price_col: Column name with actual prices.
    - forecast_col: Name of column to store forecasted prices.
    - title: Plot title.
    - zoom_days: Days to show in the zoomed-in plot.
    - figsize: Size of the plot.
    """
    df = full_df.copy()
    forecast_idx = forecast_df.index

    # Get last known price
    last_price = df.loc[forecast_idx[0] - 1, price_col]

    # Convert log returns to prices
    predicted_prices = [last_price]
    for ret in predicted_returns:
        predicted_prices.append(predicted_prices[-1] * np.exp(ret))
    predicted_prices = predicted_prices[1:]

    # Add forecast to DataFrame
    df.loc[forecast_idx, forecast_col] = predicted_prices

    # Zoomed view
    zoom_start = max(df.index[-1] - zoom_days, 0)
    df_zoom = df.iloc[zoom_start:]

    # Plot
    plt.figure(figsize=figsize)
    plt.plot(df_zoom[price_col], label="Actual Price", linewidth=2)
    plt.plot(df_zoom[forecast_col], label="Forecasted Price", linestyle="--", color="orange")
    plt.axvline(x=forecast_idx[0], color="gray", linestyle="--", label="Forecast Start")
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Price")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# Predict future returns
y_forecast_pred = trainer.predict(X_forecast)

# Plot
plot_forecast_from_predictions(
    full_df=df_full,              # full dataset with 'adj_close'
    forecast_df=forecast,        # forecast split
    predicted_returns=y_forecast_pred.ravel(),  # flatten if 2D
    price_col="adj_close",
    forecast_col="forecasted_price"
)